In [432]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from numpy import sort
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

In [433]:
#Limpieza del dataset como en TP1

oportunidades = pd.read_csv("Train_TP2_Datos_2020-2C.csv")
oportunidades['Account_Created_Date'] = pd.to_datetime(oportunidades['Account_Created_Date'])
oportunidades['Opportunity_Created_Date'] = pd.to_datetime(oportunidades['Opportunity_Created_Date'])
oportunidades['Quote_Expiry_Date'] = pd.to_datetime(oportunidades['Quote_Expiry_Date'])
oportunidades['Last_Modified_Date'] = pd.to_datetime(oportunidades['Last_Modified_Date'])
oportunidades['Planned_Delivery_Start_Date'] = pd.to_datetime(oportunidades['Planned_Delivery_Start_Date'])
oportunidades['Planned_Delivery_End_Date'] = pd.to_datetime(oportunidades['Planned_Delivery_End_Date'])
oportunidades_japon = (oportunidades.loc[oportunidades['Region'] == 'Japan'])
oportunidades = (oportunidades.loc[oportunidades['Region'] != 'Japan'])
#oportunidades_japon = oportunidades_japon.iloc[:,:3]
oportunidades_japon['Territory'] = oportunidades_japon['Territory'].replace({'None':'Japan'})
oportunidades = pd.concat([oportunidades, oportunidades_japon], axis=0)
oportunidades['Region'] = oportunidades['Region'].replace({'Japan':'APAC', 'Middle East':'EMEA'})
oportunidades[oportunidades.select_dtypes(['object']).columns] = oportunidades.select_dtypes(['object']).apply(lambda x: x.astype('category'))
oportunidades = oportunidades.loc[oportunidades['Territory'] != 'None']
oportunidades = oportunidades.loc[(oportunidades['Stage'] == 'Closed Won') | (oportunidades['Stage'] == 'Closed Lost')]
oportunidades['Tasa_De_Conversion'] = oportunidades['ASP_(converted)']/oportunidades['ASP']
oportunidades['Total_Amount'] = oportunidades['Total_Amount']*oportunidades['Tasa_De_Conversion']
oportunidades['Stage'] = oportunidades['Stage'].replace({'Closed Won': 1, 'Closed Lost': 0})
oportunidades = oportunidades.dropna()
oportunidades

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Tasa_De_Conversion
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,2015-04-21,...,2016,NaT,0,USD,8.386560e+04,USD,83865.6,1,Prod_Category_A_None,1.00000
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,2013-07-27,...,2018,NaT,25,USD,1.335719e+07,USD,13357192.5,0,Prod_Category_A_None,1.00000
5,16802,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,2013-07-27,...,2018,NaT,28,USD,1.483828e+07,USD,14838277.5,0,Prod_Category_A_None,1.00000
7,27455,Americas,NW America,1,1,1,1,0,Bureaucratic_Code_4,2015-04-21,...,2016,NaT,0,USD,5.068800e+04,USD,50688.0,1,Prod_Category_A_None,1.00000
23,28524,APAC,India,1,1,1,1,0,Bureaucratic_Code_4,2015-10-13,...,2016,NaT,3,USD,1.621851e+06,USD,1621851.0,1,Prod_Category_A_None,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16922,18519,APAC,Japan,1,1,1,1,0,Bureaucratic_Code_4,2013-07-28,...,2016,NaT,0,USD,1.320000e+04,USD,6422400.6,1,Prod_Category_A_None,1.00000
16923,27689,APAC,Japan,1,1,0,0,0,Bureaucratic_Code_4,2015-06-11,...,2016,NaT,0,JPY,1.096295e+05,JPY,12194000.0,1,Prod_Category_A_None,0.00899
16924,27688,APAC,Japan,1,1,0,0,0,Bureaucratic_Code_4,2015-06-11,...,2016,NaT,0,JPY,1.075213e+05,JPY,11959500.0,1,Prod_Category_A_None,0.00899
16925,19062,APAC,Japan,0,0,0,0,0,Bureaucratic_Code_4,2013-08-11,...,2016,NaT,0,JPY,1.576650e+05,JPY,17537000.0,1,Prod_Category_A_None,0.00899


In [434]:
#Añadimos la columna binaria de si es aprobada o no.

Requiere_Aprobacion = (oportunidades['Pricing, Delivery_Terms_Quote_Appr'] | oportunidades['Bureaucratic_Code_0_Approval'])
oportunidades.insert(3,'Requiere_Aprobacion', Requiere_Aprobacion)
oportunidades['Requiere_Aprobacion'] = oportunidades['Requiere_Aprobacion'].replace({True:1, False:0})

filtro_terminos_entrega = oportunidades['Pricing, Delivery_Terms_Quote_Appr']\
     == oportunidades['Pricing, Delivery_Terms_Approved']

filtro_codigo_burocratico = oportunidades['Bureaucratic_Code_0_Approval']\
     == oportunidades['Bureaucratic_Code_0_Approved']

oportunidades = oportunidades.drop(['Pricing, Delivery_Terms_Quote_Appr'\
    ,'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval'\
        , 'Bureaucratic_Code_0_Approved'], axis = 'columns')

oportunidades_posibles = (filtro_terminos_entrega & filtro_codigo_burocratico)

cumple_entrega = (oportunidades['Requiere_Aprobacion'] == oportunidades['Submitted_for_Approval'])
oportunidades.insert(3,'Cumple_Entrega', cumple_entrega)

oportunidades['Cumple_Entrega'] = oportunidades['Cumple_Entrega'].replace({True:1, False:0})

oportunidades.insert(3,'Es_Oportunidad_Posible', oportunidades_posibles)

oportunidades['Es_Oportunidad_Posible'] = oportunidades['Es_Oportunidad_Posible'].replace(
    {True:1, False:0})
oportunidades.head()

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Tasa_De_Conversion
2,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,2016,NaT,0,USD,83865.6,USD,83865.6,1,Prod_Category_A_None,1.0
4,16805,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,2018,NaT,25,USD,13357192.5,USD,13357192.5,0,Prod_Category_A_None,1.0
5,16802,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,2018,NaT,28,USD,14838277.5,USD,14838277.5,0,Prod_Category_A_None,1.0
7,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,2016,NaT,0,USD,50688.0,USD,50688.0,1,Prod_Category_A_None,1.0
23,28524,APAC,India,1,0,1,0,Bureaucratic_Code_4,2015-10-13,None,...,2016,NaT,3,USD,1621851.0,USD,1621851.0,1,Prod_Category_A_None,1.0


In [435]:
#Encoding de los vendedores con mean encoding respecto del total amount.
encoding_owners = oportunidades.groupby('Opportunity_Owner').agg({'Es_Oportunidad_Posible':'mean'})
maximo = encoding_owners['Es_Oportunidad_Posible'].max()
encoding_owners['Es_Oportunidad_Posible'] = encoding_owners['Es_Oportunidad_Posible']/maximo
encoding_owners.columns = ['Encoding_Vendedor']
encoding_owners = encoding_owners.reset_index()
oportunidades = oportunidades.merge(encoding_owners, how='inner', on='Opportunity_Owner')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Tasa_De_Conversion,Encoding_Vendedor
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,NaT,0,USD,8.386560e+04,USD,83865.6,1,Prod_Category_A_None,1.000000,0.790168
1,16805,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,NaT,25,USD,1.335719e+07,USD,13357192.5,0,Prod_Category_A_None,1.000000,0.790168
2,16802,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,NaT,28,USD,1.483828e+07,USD,14838277.5,0,Prod_Category_A_None,1.000000,0.790168
3,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,NaT,0,USD,5.068800e+04,USD,50688.0,1,Prod_Category_A_None,1.000000,0.790168
4,21750,Americas,NE America,1,1,0,0,Bureaucratic_Code_4,2013-07-27,Source_11,...,NaT,0,USD,1.196160e+04,USD,0.0,1,Prod_Category_A_None,1.000000,0.790168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,28141,APAC,Japan,1,1,0,0,Bureaucratic_Code_4,2015-07-03,None,...,NaT,1,JPY,3.504105e+05,JPY,38976000.0,0,Prod_Category_A_None,0.008990,1.000000
9428,28084,APAC,Japan,1,1,0,0,Bureaucratic_Code_4,2015-07-03,None,...,NaT,1,JPY,5.037161e+05,JPY,56028000.0,0,Prod_Category_A_None,0.008990,1.000000
9429,28181,APAC,Japan,1,0,1,0,Bureaucratic_Code_1,2015-07-03,None,...,NaT,0,JPY,6.887261e+04,JPY,7660800.0,0,Prod_Category_A_None,0.008990,1.000000
9430,28182,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2015-07-03,None,...,NaT,0,JPY,7.417050e+03,JPY,825000.0,1,Prod_Category_A_None,0.008990,1.000000


In [436]:
#Encoding de los territorios en base al total amount
encoding_territory = oportunidades.groupby('Territory').agg({'Total_Taxable_Amount' : 'mean'})
maximo = encoding_territory['Total_Taxable_Amount'].max()
encoding_territory['Total_Taxable_Amount'] = encoding_territory['Total_Taxable_Amount']/maximo
encoding_territory.columns = ['Encoding_Territorio']
encoding_territory = encoding_territory.reset_index()
oportunidades = oportunidades.merge(encoding_territory, how='inner', on='Territory')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Tasa_De_Conversion,Encoding_Vendedor,Encoding_Territorio
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,USD,83865.6,USD,83865.6,1,Prod_Category_A_None,1.0,0.790168,0.149057
1,16805,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,25,USD,13357192.5,USD,13357192.5,0,Prod_Category_A_None,1.0,0.790168,0.149057
2,16802,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,28,USD,14838277.5,USD,14838277.5,0,Prod_Category_A_None,1.0,0.790168,0.149057
3,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,USD,50688.0,USD,50688.0,1,Prod_Category_A_None,1.0,0.790168,0.149057
4,21527,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2013-07-27,Source_11,...,0,USD,2814.0,USD,2814.0,1,Prod_Category_A_None,1.0,0.790168,0.149057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,14862,APAC,New Zealand,0,0,1,0,Bureaucratic_Code_5,2017-05-23,Source_7,...,0,USD,28875.0,USD,147262.5,1,Prod_Category_A_None,1.0,0.726316,0.002388
9428,14868,APAC,New Zealand,0,0,1,0,Bureaucratic_Code_5,2017-05-23,Source_7,...,0,USD,28875.0,USD,147262.5,1,Prod_Category_A_None,1.0,0.726316,0.002388
9429,14873,APAC,New Zealand,0,0,1,0,Bureaucratic_Code_5,2017-05-23,Source_7,...,0,USD,28875.0,USD,147262.5,1,Prod_Category_A_None,1.0,0.726316,0.002388
9430,14879,APAC,New Zealand,0,0,1,0,Bureaucratic_Code_5,2017-05-23,Source_7,...,0,USD,28875.0,USD,147262.5,1,Prod_Category_A_None,1.0,0.726316,0.002388


In [437]:
#Encoding de las regiones en base a Dummies

#encoding_region = oportunidades.groupby('Region').agg({'Total_Amount' : 'mean'})
#encoding_region.columns = ['Encoding_Region']
#encoding_region = encoding_region.reset_index()
#oportunidades = oportunidades.merge(encoding_region, how='inner', on='Region')
oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Region'])],axis='columns')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Tasa_De_Conversion,Encoding_Vendedor,Encoding_Territorio,APAC,Americas,EMEA
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,USD,83865.6,1,Prod_Category_A_None,1.0,0.790168,0.149057,0,1,0
1,16805,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,USD,13357192.5,0,Prod_Category_A_None,1.0,0.790168,0.149057,0,1,0
2,16802,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,USD,14838277.5,0,Prod_Category_A_None,1.0,0.790168,0.149057,0,1,0
3,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,USD,50688.0,1,Prod_Category_A_None,1.0,0.790168,0.149057,0,1,0
4,21527,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2013-07-27,Source_11,...,USD,2814.0,1,Prod_Category_A_None,1.0,0.790168,0.149057,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,14862,APAC,New Zealand,0,0,1,0,Bureaucratic_Code_5,2017-05-23,Source_7,...,USD,147262.5,1,Prod_Category_A_None,1.0,0.726316,0.002388,1,0,0
9428,14868,APAC,New Zealand,0,0,1,0,Bureaucratic_Code_5,2017-05-23,Source_7,...,USD,147262.5,1,Prod_Category_A_None,1.0,0.726316,0.002388,1,0,0
9429,14873,APAC,New Zealand,0,0,1,0,Bureaucratic_Code_5,2017-05-23,Source_7,...,USD,147262.5,1,Prod_Category_A_None,1.0,0.726316,0.002388,1,0,0
9430,14879,APAC,New Zealand,0,0,1,0,Bureaucratic_Code_5,2017-05-23,Source_7,...,USD,147262.5,1,Prod_Category_A_None,1.0,0.726316,0.002388,1,0,0


In [438]:
#Encoding del tipo de moneda en base al mean del ASP
encoding_moneda = oportunidades.groupby('Total_Amount_Currency').agg({'Tasa_De_Conversion' : 'mean'})
maximo = encoding_moneda['Tasa_De_Conversion'].max()
encoding_moneda.columns = ['Encoding_Moneda']
encoding_moneda = encoding_moneda.reset_index()
oportunidades = oportunidades.merge(encoding_moneda, how='inner', on='Total_Amount_Currency')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Total_Taxable_Amount,Stage,Prod_Category_A,Tasa_De_Conversion,Encoding_Vendedor,Encoding_Territorio,APAC,Americas,EMEA,Encoding_Moneda
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,83865.6,1,Prod_Category_A_None,1.000000,0.790168,0.149057,0,1,0,1.000000
1,16805,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,13357192.5,0,Prod_Category_A_None,1.000000,0.790168,0.149057,0,1,0,1.000000
2,16802,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,14838277.5,0,Prod_Category_A_None,1.000000,0.790168,0.149057,0,1,0,1.000000
3,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,50688.0,1,Prod_Category_A_None,1.000000,0.790168,0.149057,0,1,0,1.000000
4,21527,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2013-07-27,Source_11,...,2814.0,1,Prod_Category_A_None,1.000000,0.790168,0.149057,0,1,0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,17555,APAC,Australia,1,0,1,0,Bureaucratic_Code_4,2013-10-03,None,...,117600.0,1,Prod_Category_A_None,0.707617,0.916667,0.053931,1,0,0,0.707613
9428,18413,APAC,Australia,1,0,1,0,Bureaucratic_Code_4,2013-07-28,None,...,117600.0,1,Prod_Category_A_None,0.707617,0.916667,0.053931,1,0,0,0.707613
9429,18416,APAC,Australia,1,0,1,0,Bureaucratic_Code_4,2013-07-28,None,...,117600.0,1,Prod_Category_A_None,0.707617,0.916667,0.053931,1,0,0,0.707613
9430,22188,APAC,Australia,1,0,1,0,Bureaucratic_Code_4,2014-09-19,Source_9,...,0.0,0,Prod_Category_A_None,0.707625,0.916667,0.053931,1,0,0,0.707613


In [439]:
#Encoding del Bureaucratic_Code en base a Dummies
oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Bureaucratic_Code'])], axis='columns')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Americas,EMEA,Encoding_Moneda,Bureaucratic_Code_0,Bureaucratic_Code_1,Bureaucratic_Code_2,Bureaucratic_Code_3,Bureaucratic_Code_4,Bureaucratic_Code_5,Bureaucratic_Code_6
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,1,0,1.000000,0,0,0,0,1,0,0
1,16805,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,1,0,1.000000,0,0,0,0,0,1,0
2,16802,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,1,0,1.000000,0,0,0,0,0,1,0
3,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,1,0,1.000000,0,0,0,0,1,0,0
4,21527,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2013-07-27,Source_11,...,1,0,1.000000,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,17555,APAC,Australia,1,0,1,0,Bureaucratic_Code_4,2013-10-03,None,...,0,0,0.707613,0,0,0,0,1,0,0
9428,18413,APAC,Australia,1,0,1,0,Bureaucratic_Code_4,2013-07-28,None,...,0,0,0.707613,0,0,0,0,1,0,0
9429,18416,APAC,Australia,1,0,1,0,Bureaucratic_Code_4,2013-07-28,None,...,0,0,0.707613,0,0,0,0,1,0,0
9430,22188,APAC,Australia,1,0,1,0,Bureaucratic_Code_4,2014-09-19,Source_9,...,0,0,0.707613,0,0,0,0,1,0,0


In [440]:
#Encoding del Billing Country en base a mean de Total_Amount.
encoding_bill_c = oportunidades.groupby('Billing_Country').agg({'Total_Amount' : 'mean'})
maximo = encoding_bill_c['Total_Amount'].max()
encoding_bill_c['Total_Amount'] = encoding_bill_c['Total_Amount']/maximo
encoding_bill_c.columns = ['Encoding_Billing_Country']
encoding_bill_c = encoding_bill_c.reset_index()
oportunidades = oportunidades.merge(encoding_bill_c, how='inner', on='Billing_Country')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,EMEA,Encoding_Moneda,Bureaucratic_Code_0,Bureaucratic_Code_1,Bureaucratic_Code_2,Bureaucratic_Code_3,Bureaucratic_Code_4,Bureaucratic_Code_5,Bureaucratic_Code_6,Encoding_Billing_Country
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,1.000000,0,0,0,0,1,0,0,0.056493
1,16805,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,0,1.000000,0,0,0,0,0,1,0,0.056493
2,16802,Americas,NW America,0,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,0,1.000000,0,0,0,0,0,1,0,0.056493
3,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,1.000000,0,0,0,0,1,0,0,0.056493
4,21527,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2013-07-27,Source_11,...,0,1.000000,0,0,0,0,1,0,0,0.056493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,6773,EMEA,Malta,1,0,1,0,Bureaucratic_Code_4,2017-07-13,Source_13,...,1,1.131058,0,0,0,0,1,0,0,0.117869
9428,6779,EMEA,Malta,1,0,1,0,Bureaucratic_Code_4,2017-07-13,Source_13,...,1,1.131058,0,0,0,0,1,0,0,0.117869
9429,6790,EMEA,Malta,1,0,1,0,Bureaucratic_Code_4,2017-07-13,Source_13,...,1,1.131058,0,0,0,0,1,0,0,0.117869
9430,6762,EMEA,Malta,1,0,1,0,Bureaucratic_Code_4,2017-07-13,Source_13,...,1,1.131058,0,0,0,0,1,0,0,0.117869


In [441]:
#Encoding del Product_Family en base a mean de Total_Amount.
encoding_product_family = oportunidades.groupby('Product_Family').agg({'Total_Amount' : 'mean'})
maximo = encoding_product_family['Total_Amount'].max()
encoding_product_family['Total_Amount'] = encoding_product_family['Total_Amount']/maximo
encoding_product_family.columns = ['Encoding_Prod_Family']
encoding_product_family = encoding_product_family.reset_index()
oportunidades = oportunidades.merge(encoding_product_family, how='inner', on='Product_Family')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Encoding_Moneda,Bureaucratic_Code_0,Bureaucratic_Code_1,Bureaucratic_Code_2,Bureaucratic_Code_3,Bureaucratic_Code_4,Bureaucratic_Code_5,Bureaucratic_Code_6,Encoding_Billing_Country,Encoding_Prod_Family
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,1.000000,0,0,0,0,1,0,0,0.056493,0.001195
1,27442,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,1.000000,0,0,0,0,1,0,0,0.056493,0.001195
2,15713,EMEA,Germany,1,0,1,0,Bureaucratic_Code_4,2013-07-27,None,...,1.131058,0,0,0,0,1,0,0,0.023946,0.001195
3,16744,EMEA,Spain,1,0,1,0,Bureaucratic_Code_4,2013-07-27,Source_11,...,1.131058,0,0,0,0,1,0,0,0.032617,0.001195
4,15045,APAC,India,0,0,1,0,Bureaucratic_Code_5,2017-05-24,Source_7,...,1.000000,0,0,0,0,0,1,0,0.024566,0.001195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,9753,APAC,China (PRC),1,0,1,0,Bureaucratic_Code_4,2016-01-29,None,...,1.000000,0,0,0,0,1,0,0,0.002731,0.000037
9428,9992,APAC,China (PRC),1,0,1,0,Bureaucratic_Code_4,2016-01-29,None,...,1.000000,0,0,0,0,1,0,0,0.002731,0.000006
9429,9859,APAC,China (PRC),1,0,1,0,Bureaucratic_Code_4,2016-01-29,None,...,1.000000,0,0,0,0,1,0,0,0.002731,0.000039
9430,9163,APAC,China (PRC),1,0,1,0,Bureaucratic_Code_4,2016-01-29,None,...,1.000000,0,0,0,0,1,0,0,0.002731,0.000059


In [442]:
#Encoding con Dummies de Account_Type

oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Account_Type'])],axis='columns')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Bureaucratic_Code_6,Encoding_Billing_Country,Encoding_Prod_Family,Account_Type_0,Account_Type_1,Account_Type_2,Account_Type_4,Account_Type_5,Account_Type_6,None
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0.056493,0.001195,0,0,0,0,1,0,0
1,27442,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0.056493,0.001195,0,0,0,0,1,0,0
2,15713,EMEA,Germany,1,0,1,0,Bureaucratic_Code_4,2013-07-27,None,...,0,0.023946,0.001195,1,0,0,0,0,0,0
3,16744,EMEA,Spain,1,0,1,0,Bureaucratic_Code_4,2013-07-27,Source_11,...,0,0.032617,0.001195,1,0,0,0,0,0,0
4,15045,APAC,India,0,0,1,0,Bureaucratic_Code_5,2017-05-24,Source_7,...,0,0.024566,0.001195,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,9753,APAC,China (PRC),1,0,1,0,Bureaucratic_Code_4,2016-01-29,None,...,0,0.002731,0.000037,1,0,0,0,0,0,0
9428,9992,APAC,China (PRC),1,0,1,0,Bureaucratic_Code_4,2016-01-29,None,...,0,0.002731,0.000006,1,0,0,0,0,0,0
9429,9859,APAC,China (PRC),1,0,1,0,Bureaucratic_Code_4,2016-01-29,None,...,0,0.002731,0.000039,1,0,0,0,0,0,0
9430,9163,APAC,China (PRC),1,0,1,0,Bureaucratic_Code_4,2016-01-29,None,...,0,0.002731,0.000059,1,0,0,0,0,0,0


In [443]:
#Encoding del Account Name en base al mean del Total_Amount debido a que son muchos nunique

encoding_account_name = oportunidades.groupby('Account_Name').agg({'Encoding_Territorio' : 'mean'})
maximo = encoding_account_name['Encoding_Territorio'].max()
encoding_account_name['Encoding_Territorio'] = encoding_account_name['Encoding_Territorio']/maximo
encoding_account_name.columns = ['Encoding_Acc_Name']
encoding_account_name = encoding_account_name.reset_index()
oportunidades = oportunidades.merge(encoding_account_name, how='inner', on='Account_Name')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Encoding_Billing_Country,Encoding_Prod_Family,Account_Type_0,Account_Type_1,Account_Type_2,Account_Type_4,Account_Type_5,Account_Type_6,None,Encoding_Acc_Name
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0.056493,0.001195,0,0,0,0,1,0,0,0.149057
1,27442,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0.056493,0.001195,0,0,0,0,1,0,0,0.149057
2,27447,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0.056493,0.080286,0,0,0,0,1,0,0,0.149057
3,27445,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0.056493,0.080286,0,0,0,0,1,0,0,0.149057
4,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0.056493,0.001388,0,0,0,0,1,0,0,0.149057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,18523,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0.011539,0.043723,0,0,1,0,0,0,0,0.206464
9428,18513,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0.011539,0.043723,0,0,1,0,0,0,0,0.206464
9429,18519,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0.011539,0.000120,0,0,1,0,0,0,0,0.206464
9430,18516,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0.011539,0.000230,0,0,1,0,0,0,0,0.206464


In [444]:
#Encoding de Delivery Terms en base a Dummies
oportunidades = pd.concat([oportunidades,pd.get_dummies(oportunidades['Delivery_Terms'])],axis='columns')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Encoding_Acc_Name,Delivery_Terms_0,Delivery_Terms_1,Delivery_Terms_2,Delivery_Terms_3,Delivery_Terms_4,Delivery_Terms_5,Delivery_Terms_6,Delivery_Terms_7,Delivery_Terms_8
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0.149057,0,0,0,0,1,0,0,0,0
1,27442,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0.149057,0,0,0,0,1,0,0,0,0
2,27447,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0.149057,0,0,0,0,0,0,0,0,1
3,27445,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0.149057,0,0,0,0,0,0,0,0,1
4,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0.149057,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,18523,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0.206464,0,0,0,0,0,0,1,0,0
9428,18513,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0.206464,0,0,0,0,0,0,1,0,0
9429,18519,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0.206464,0,0,0,0,0,0,1,0,0
9430,18516,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0.206464,0,0,0,0,0,0,1,0,0


In [445]:
#Obtenemos el año de creación de la oportunidad como columna, y el mes también.
oportunidades['Opportunity_Created_Year'] = oportunidades['Opportunity_Created_Date'].dt.year
oportunidades['Opportunity_Created_Month'] = oportunidades['Opportunity_Created_Date'].dt.month
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Delivery_Terms_1,Delivery_Terms_2,Delivery_Terms_3,Delivery_Terms_4,Delivery_Terms_5,Delivery_Terms_6,Delivery_Terms_7,Delivery_Terms_8,Opportunity_Created_Year,Opportunity_Created_Month
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,1,0,0,0,0,2015,12
1,27442,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,1,0,0,0,0,2016,4
2,27447,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,0,0,0,0,1,2017,11
3,27445,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,0,0,0,0,1,2017,12
4,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,1,0,0,0,0,2015,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,18523,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0,0,0,0,0,1,0,0,2016,3
9428,18513,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0,0,0,0,0,1,0,0,2015,12
9429,18519,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0,0,0,0,0,1,0,0,2015,12
9430,18516,APAC,Japan,1,0,1,0,Bureaucratic_Code_4,2013-07-28,Source_7,...,0,0,0,0,0,1,0,0,2015,12


In [446]:
#Encoding de oportunidades['Opportunity_Type'] en base al mean del Total_Taxable_Amount

encoding_op_type = oportunidades.groupby('Opportunity_Type').agg({'Total_Taxable_Amount' : 'mean'})
maximo = encoding_op_type['Total_Taxable_Amount'].max()
encoding_op_type['Total_Taxable_Amount'] = encoding_op_type['Total_Taxable_Amount']/maximo
encoding_op_type.columns = ['Encoding_Opportunity_Type']
encoding_op_type = encoding_op_type.reset_index()
oportunidades = oportunidades.merge(encoding_op_type, how='inner', on='Opportunity_Type')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Delivery_Terms_2,Delivery_Terms_3,Delivery_Terms_4,Delivery_Terms_5,Delivery_Terms_6,Delivery_Terms_7,Delivery_Terms_8,Opportunity_Created_Year,Opportunity_Created_Month,Encoding_Opportunity_Type
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,1,0,0,0,0,2015,12,0.177423
1,27447,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,0,0,0,1,2017,11,0.177423
2,27445,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,0,0,0,1,2017,12,0.177423
3,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,1,0,0,0,0,2015,12,0.177423
4,27468,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,1,0,0,0,0,2016,7,0.177423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,7075,EMEA,Germany,1,1,0,0,Bureaucratic_Code_4,2018-08-24,Source_13,...,1,0,0,0,0,0,0,2018,8,0.037647
9428,7086,EMEA,Germany,1,1,0,0,Bureaucratic_Code_4,2018-08-24,Source_13,...,1,0,0,0,0,0,0,2018,8,0.037647
9429,28068,EMEA,Germany,1,0,1,0,Bureaucratic_Code_4,2015-07-02,Source_9,...,1,0,0,0,0,0,0,2018,10,0.037647
9430,13901,Americas,Canada,0,0,1,0,Bureaucratic_Code_5,2017-05-20,Source_7,...,0,0,0,0,0,0,0,2018,1,0.027789


In [447]:
#Get Dummies de Source para utilizar los mas usados (7, 9, 11, 13, 3)

oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Source '])],axis = 'columns')
oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Cumple_Entrega,Requiere_Aprobacion,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Source_11,Source_12,Source_13,Source_2,Source_3,Source_4,Source_5,Source_6,Source_7,Source_9
0,27446,Americas,NW America,1,1,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,0,0,0,0,0,1,0
1,27447,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,0,0,0,0,0,1,0
2,27445,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,0,0,0,0,0,1,0
3,27455,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,0,0,0,0,0,1,0
4,27468,Americas,NW America,1,0,1,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,7075,EMEA,Germany,1,1,0,0,Bureaucratic_Code_4,2018-08-24,Source_13,...,0,0,1,0,0,0,0,0,0,0
9428,7086,EMEA,Germany,1,1,0,0,Bureaucratic_Code_4,2018-08-24,Source_13,...,0,0,1,0,0,0,0,0,0,0
9429,28068,EMEA,Germany,1,0,1,0,Bureaucratic_Code_4,2015-07-02,Source_9,...,0,0,0,0,0,0,0,0,0,1
9430,13901,Americas,Canada,0,0,1,0,Bureaucratic_Code_5,2017-05-20,Source_7,...,0,0,0,0,0,0,0,0,1,0


In [448]:
#Una vez que tenemos el dataset limpio como en el TP1, iniciamos con el uso de XGBoost como algoritmo de ML.

#Parametros resultantes del tuning:{'objective': 'reg:logistic', 'max_depth': 7, 'n_estimators': 140, 
#'colsample_bytree': 0.9, 'subsample': 0.9, 'learning_rate': 0.085, 'alpha': 0.01, 'gamma': 0.2}

#Con Error: 0.3574255624906347

params = {
    'objective':'binary:logistic',
    'max_depth':12,
    'n_estimators':450,
    'colsample_bytree':0.3,
    'subsample':0.7,
    'learning_rate':0.014,
    'alpha':0.01,
    'gamma':0.0
}

params2 = {
    'objective':'binary:logistic',
    'max_depth':9,
    'n_estimators':350,
    'colsample_bytree':0.5,
    'subsample':0.5,
    'learning_rate':0.02,
    'alpha':0.0001,
    'gamma':0.1
}

xg_reg = xgb.XGBRegressor(objective =params['objective'], 
                colsample_bytree = params['colsample_bytree'], learning_rate = params['learning_rate'],
                max_depth = params['max_depth'], alpha = params['alpha'], n_estimators = params['n_estimators'], 
                subsample = params['subsample'], eval_metric = 'logloss')

xg_reg2 = xgb.XGBRegressor(objective =params2['objective'], 
                colsample_bytree = params2['colsample_bytree'], learning_rate = params2['learning_rate'],
                max_depth = params2['max_depth'], alpha = params2['alpha'], n_estimators = params2['n_estimators'], 
                subsample = params2['subsample'], eval_metric = 'logloss')

std_deviation_amounts = oportunidades.groupby('Opportunity_ID').agg({'TRF':'std', 'Opportunity_Created_Month':'std'})
std_deviation_amounts.columns = ['TRF_Std', 'Opp_Created_Month_Std']
std_deviation_amounts = std_deviation_amounts.reset_index()

#Solo usamos Bureau_Code_ 0,1,2,4,5 porque son los que tiene el set de test.
y_train = oportunidades.groupby('Opportunity_ID').agg({"Stage":'max'})
x_train = oportunidades.groupby('Opportunity_ID').agg({'Opportunity_Created_Year' : lambda x: pd.Series.mode(x)[0],'Opportunity_Created_Month' : lambda x: pd.Series.mode(x)[0],
                                                      'Encoding_Moneda':lambda x: pd.Series.mode(x)[0], 'Total_Amount': 'sum',
                                                      'Total_Taxable_Amount':'sum','Encoding_Territorio':lambda x: pd.Series.mode(x)[0],'Encoding_Vendedor':lambda x: pd.Series.mode(x)[0],\
                                                      'Es_Oportunidad_Posible':'mean', 'TRF':'mean','Encoding_Billing_Country':lambda x: pd.Series.mode(x)[0],\
                                                       'Encoding_Prod_Family':lambda x: pd.Series.mode(x)[0], 'Encoding_Acc_Name':lambda x: pd.Series.mode(x)[0],\
                                                      'Encoding_Opportunity_Type':lambda x: pd.Series.mode(x)[0],'APAC' : 'mean','Americas':'mean','EMEA':'mean',\
                                                       'Cumple_Entrega':'mean'
                                                        })

#'Bureaucratic_Code_0':'mean','Bureaucratic_Code_1':'mean','Bureaucratic_Code_2':'mean','Bureaucratic_Code_4':'mean','Bureaucratic_Code_5':'mean',
#'Delivery_Terms_0':'mean','Delivery_Terms_1':'mean','Delivery_Terms_2':'mean','Delivery_Terms_3':'mean','Delivery_Terms_4':'mean','Delivery_Terms_5':'mean','Delivery_Terms_6':'mean','Delivery_Terms_7':'mean','Delivery_Terms_8':'mean',
#'Account_Type_0':'mean','Account_Type_1':'mean','Account_Type_2':'mean','Account_Type_4':'mean', 'Account_Type_5':'mean',
#'Source_3':'mean', 'Source_7':'mean','Source_9':'mean','Source_11':'mean','Source_13':'mean'

x_train = x_train.reset_index()
x_train = x_train.merge(std_deviation_amounts, on='Opportunity_ID', how='inner')
x_train = x_train.set_index('Opportunity_ID')
#y_train = x_train['Stage']
#x_train = x_train.drop('Stage', axis='columns')
x_train['TRF_Std'] = x_train['TRF_Std'].replace({np.nan:0})
x_train['Opp_Created_Month_Std'] = x_train['Opp_Created_Month_Std'].replace({np.nan:0})

division_x_train, division_x_test, division_y_train, division_y_test = train_test_split(x_train, y_train, test_size = 0.3,\
                                                                                       random_state = 123)

In [449]:
#Entrenamos al xgboost
#xg_reg.fit(division_x_train, division_y_train)

data_dmatrix = xgb.DMatrix(data=x_train,label=y_train)

xg_reg.fit(division_x_train, division_y_train)
xg_reg2.fit(division_x_train, division_y_train)

#xg_reg = xgb.train(params=params, dtrain = data_dmatrix, num_boost_round=2000)

#xg_reg2 = xgb.train(params=params2, dtrain = data_dmatrix, num_boost_round=2000)

#xg_reg2.fit(division_x_train, division_y_train)

XGBRegressor(alpha=0.0001, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             eval_metric='logloss', gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.02, max_delta_step=0,
             max_depth=9, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=350, n_jobs=12,
             num_parallel_tree=1, objective='binary:logistic', random_state=0,
             reg_alpha=9.99999975e-05, reg_lambda=1, scale_pos_weight=1,
             subsample=0.5, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [450]:
#Con 1er modelo

predictions = xg_reg.predict(division_x_test)
accuracy = accuracy_score(division_y_test, predictions.round())
log_loss_x = log_loss(division_y_test, predictions)
print("xg_reg -> n=%d, Accuracy: %.2f%%" % (division_x_train.shape[1], accuracy*100.0) + ", Log Loss: " + str(log_loss_x))

xg_reg -> n=19, Accuracy: 79.84%, Log Loss: 0.4233371176831852


In [451]:
cv_results = xgb.cv(dtrain=data_dmatrix, params=params2, nfold=10,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="logloss", as_pandas=True, seed=123)
cv_results

[18:34:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:34:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:34:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in la

,train-logloss-mean,train-logloss-std,test-logloss-mean,test-logloss-std
0,0.684705,0.000118,0.685788,0.000444
1,0.677144,0.000596,0.679113,0.000894
2,0.669869,0.001003,0.672752,0.000998
3,0.663063,0.001301,0.666683,0.001983
4,0.656228,0.001327,0.660754,0.001924
5,0.649441,0.001825,0.654890,0.002293
6,0.642833,0.002239,0.649027,0.002611
7,0.636446,0.002502,0.643481,0.002878
8,0.630496,0.002575,0.638371,0.003258
9,0.624548,0.002447,0.633155,0.003204


In [452]:
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=10,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="logloss", as_pandas=True, seed=123)
cv_results

[18:34:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:34:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:34:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in la

,train-logloss-mean,train-logloss-std,test-logloss-mean,test-logloss-std
0,0.686906,0.000103,0.688102,0.000263
1,0.681789,0.000601,0.683610,0.000501
2,0.676722,0.001201,0.679244,0.000933
3,0.672371,0.001165,0.675426,0.001242
4,0.667719,0.001275,0.671574,0.001468
5,0.663269,0.001650,0.667606,0.001728
6,0.658864,0.002403,0.663899,0.002167
7,0.654830,0.002353,0.660522,0.002256
8,0.650744,0.002096,0.656966,0.002360
9,0.646593,0.002039,0.653419,0.002337


In [453]:
#Con 2do modelo

predictions2 = xg_reg2.predict(division_x_test)
accuracy = accuracy_score(division_y_test, predictions2.round())
log_loss_x = log_loss(division_y_test, predictions2)
print("xg_reg2 -> n=%d, Accuracy: %.2f%%" % (division_x_train.shape[1], accuracy*100.0) + ", Log Loss: " + str(log_loss_x))

xg_reg2 -> n=19, Accuracy: 80.03%, Log Loss: 0.42111453417776923


In [454]:
#Con el promedio

predictions_final = (predictions+predictions2)/2
accuracy = accuracy_score(division_y_test, predictions_final.round())
log_loss_x = log_loss(division_y_test, predictions_final)
print("Promedio -> n=%d, Accuracy: %.2f%%" % (division_x_train.shape[1], accuracy*100.0) + ", Log Loss: " + str(log_loss_x))

Promedio -> n=19, Accuracy: 80.12%, Log Loss: 0.42052349946180617


In [455]:
cat = CatBoostRegressor()
cat.fit(division_x_train, division_y_train)
accuracy = accuracy_score(division_y_test, cat.predict(division_x_test).round())
log_loss_x = log_loss(division_y_test, cat.predict(division_x_test))
print("CatBoost -> n=%d, Accuracy: %.2f%%" % (division_x_train.shape[1], accuracy*100.0) + ", Log Loss: " + str(log_loss_x))

Learning rate set to 0.051738
0:	learn: 0.4865819	total: 4.73ms	remaining: 4.72s
1:	learn: 0.4798825	total: 7.91ms	remaining: 3.95s
2:	learn: 0.4737537	total: 11.3ms	remaining: 3.76s
3:	learn: 0.4673982	total: 14.1ms	remaining: 3.52s
4:	learn: 0.4617801	total: 16.7ms	remaining: 3.31s
5:	learn: 0.4564856	total: 19.8ms	remaining: 3.28s
6:	learn: 0.4516011	total: 22.5ms	remaining: 3.19s
7:	learn: 0.4467099	total: 25.3ms	remaining: 3.14s
8:	learn: 0.4425982	total: 28.1ms	remaining: 3.1s
9:	learn: 0.4388111	total: 30.6ms	remaining: 3.03s
10:	learn: 0.4347821	total: 33.6ms	remaining: 3.02s
11:	learn: 0.4313478	total: 36.2ms	remaining: 2.98s
12:	learn: 0.4282842	total: 39ms	remaining: 2.96s
13:	learn: 0.4256890	total: 41.5ms	remaining: 2.92s
14:	learn: 0.4230313	total: 44ms	remaining: 2.89s
15:	learn: 0.4203052	total: 46.6ms	remaining: 2.87s
16:	learn: 0.4179592	total: 49.5ms	remaining: 2.86s
17:	learn: 0.4158979	total: 52.2ms	remaining: 2.85s
18:	learn: 0.4139760	total: 54.8ms	remaining: 2.8

177:	learn: 0.3543106	total: 503ms	remaining: 2.32s
178:	learn: 0.3542174	total: 506ms	remaining: 2.32s
179:	learn: 0.3540514	total: 509ms	remaining: 2.32s
180:	learn: 0.3537815	total: 511ms	remaining: 2.31s
181:	learn: 0.3536657	total: 514ms	remaining: 2.31s
182:	learn: 0.3532980	total: 517ms	remaining: 2.31s
183:	learn: 0.3531610	total: 519ms	remaining: 2.3s
184:	learn: 0.3530987	total: 522ms	remaining: 2.3s
185:	learn: 0.3529513	total: 524ms	remaining: 2.29s
186:	learn: 0.3527017	total: 527ms	remaining: 2.29s
187:	learn: 0.3525673	total: 530ms	remaining: 2.29s
188:	learn: 0.3524524	total: 533ms	remaining: 2.29s
189:	learn: 0.3522096	total: 536ms	remaining: 2.28s
190:	learn: 0.3519791	total: 539ms	remaining: 2.28s
191:	learn: 0.3517608	total: 541ms	remaining: 2.28s
192:	learn: 0.3515452	total: 544ms	remaining: 2.27s
193:	learn: 0.3512851	total: 547ms	remaining: 2.27s
194:	learn: 0.3511860	total: 550ms	remaining: 2.27s
195:	learn: 0.3509844	total: 553ms	remaining: 2.27s
196:	learn: 0.

349:	learn: 0.3273914	total: 1.01s	remaining: 1.88s
350:	learn: 0.3272181	total: 1.01s	remaining: 1.87s
351:	learn: 0.3270634	total: 1.01s	remaining: 1.87s
352:	learn: 0.3270231	total: 1.02s	remaining: 1.86s
353:	learn: 0.3269421	total: 1.02s	remaining: 1.86s
354:	learn: 0.3267050	total: 1.02s	remaining: 1.86s
355:	learn: 0.3265580	total: 1.03s	remaining: 1.86s
356:	learn: 0.3264088	total: 1.03s	remaining: 1.85s
357:	learn: 0.3262682	total: 1.03s	remaining: 1.85s
358:	learn: 0.3261503	total: 1.03s	remaining: 1.85s
359:	learn: 0.3259780	total: 1.04s	remaining: 1.84s
360:	learn: 0.3257901	total: 1.04s	remaining: 1.84s
361:	learn: 0.3256118	total: 1.04s	remaining: 1.84s
362:	learn: 0.3254917	total: 1.05s	remaining: 1.84s
363:	learn: 0.3253251	total: 1.05s	remaining: 1.83s
364:	learn: 0.3251835	total: 1.05s	remaining: 1.83s
365:	learn: 0.3249832	total: 1.06s	remaining: 1.83s
366:	learn: 0.3248629	total: 1.06s	remaining: 1.83s
367:	learn: 0.3247236	total: 1.06s	remaining: 1.82s
368:	learn: 

528:	learn: 0.3068952	total: 1.52s	remaining: 1.36s
529:	learn: 0.3068190	total: 1.53s	remaining: 1.35s
530:	learn: 0.3067168	total: 1.53s	remaining: 1.35s
531:	learn: 0.3066365	total: 1.53s	remaining: 1.35s
532:	learn: 0.3065779	total: 1.54s	remaining: 1.35s
533:	learn: 0.3063636	total: 1.54s	remaining: 1.34s
534:	learn: 0.3062970	total: 1.54s	remaining: 1.34s
535:	learn: 0.3062394	total: 1.54s	remaining: 1.34s
536:	learn: 0.3061074	total: 1.55s	remaining: 1.33s
537:	learn: 0.3060373	total: 1.55s	remaining: 1.33s
538:	learn: 0.3059535	total: 1.55s	remaining: 1.33s
539:	learn: 0.3058096	total: 1.56s	remaining: 1.33s
540:	learn: 0.3057169	total: 1.56s	remaining: 1.32s
541:	learn: 0.3055958	total: 1.56s	remaining: 1.32s
542:	learn: 0.3055009	total: 1.57s	remaining: 1.32s
543:	learn: 0.3053690	total: 1.57s	remaining: 1.31s
544:	learn: 0.3052565	total: 1.57s	remaining: 1.31s
545:	learn: 0.3051901	total: 1.57s	remaining: 1.31s
546:	learn: 0.3050720	total: 1.58s	remaining: 1.31s
547:	learn: 

705:	learn: 0.2892046	total: 2.03s	remaining: 846ms
706:	learn: 0.2890739	total: 2.03s	remaining: 843ms
707:	learn: 0.2889783	total: 2.04s	remaining: 840ms
708:	learn: 0.2888971	total: 2.04s	remaining: 837ms
709:	learn: 0.2887975	total: 2.04s	remaining: 834ms
710:	learn: 0.2887932	total: 2.04s	remaining: 831ms
711:	learn: 0.2887025	total: 2.05s	remaining: 828ms
712:	learn: 0.2885606	total: 2.05s	remaining: 825ms
713:	learn: 0.2884598	total: 2.05s	remaining: 822ms
714:	learn: 0.2884118	total: 2.06s	remaining: 820ms
715:	learn: 0.2883021	total: 2.06s	remaining: 817ms
716:	learn: 0.2882958	total: 2.06s	remaining: 814ms
717:	learn: 0.2881790	total: 2.06s	remaining: 811ms
718:	learn: 0.2880655	total: 2.07s	remaining: 808ms
719:	learn: 0.2879724	total: 2.07s	remaining: 806ms
720:	learn: 0.2879307	total: 2.07s	remaining: 803ms
721:	learn: 0.2878092	total: 2.08s	remaining: 800ms
722:	learn: 0.2877000	total: 2.08s	remaining: 797ms
723:	learn: 0.2875987	total: 2.08s	remaining: 794ms
724:	learn: 

888:	learn: 0.2752961	total: 2.54s	remaining: 317ms
889:	learn: 0.2752203	total: 2.54s	remaining: 314ms
890:	learn: 0.2751941	total: 2.55s	remaining: 312ms
891:	learn: 0.2751452	total: 2.55s	remaining: 309ms
892:	learn: 0.2750927	total: 2.55s	remaining: 306ms
893:	learn: 0.2750083	total: 2.55s	remaining: 303ms
894:	learn: 0.2748752	total: 2.56s	remaining: 300ms
895:	learn: 0.2747951	total: 2.56s	remaining: 297ms
896:	learn: 0.2747466	total: 2.56s	remaining: 294ms
897:	learn: 0.2747139	total: 2.56s	remaining: 291ms
898:	learn: 0.2746322	total: 2.57s	remaining: 289ms
899:	learn: 0.2745231	total: 2.57s	remaining: 286ms
900:	learn: 0.2744568	total: 2.57s	remaining: 283ms
901:	learn: 0.2743597	total: 2.58s	remaining: 280ms
902:	learn: 0.2742578	total: 2.58s	remaining: 277ms
903:	learn: 0.2742052	total: 2.58s	remaining: 274ms
904:	learn: 0.2741272	total: 2.58s	remaining: 271ms
905:	learn: 0.2741071	total: 2.59s	remaining: 268ms
906:	learn: 0.2739897	total: 2.59s	remaining: 266ms
907:	learn: 

In [456]:
xg_reg.fit(x_train, y_train)
xg_reg2.fit(x_train, y_train)

XGBRegressor(alpha=0.0001, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             eval_metric='logloss', gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.02, max_delta_step=0,
             max_depth=9, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=350, n_jobs=12,
             num_parallel_tree=1, objective='binary:logistic', random_state=0,
             reg_alpha=9.99999975e-05, reg_lambda=1, scale_pos_weight=1,
             subsample=0.5, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [457]:
#Limpieza del dataset de test

oportunidades = pd.read_csv("Test_TP2_Datos_2020-2C.csv")
oportunidades['Account_Created_Date'] = pd.to_datetime(oportunidades['Account_Created_Date'])
oportunidades['Opportunity_Created_Date'] = pd.to_datetime(oportunidades['Opportunity_Created_Date'])
oportunidades['Quote_Expiry_Date'] = pd.to_datetime(oportunidades['Quote_Expiry_Date'])
oportunidades['Last_Modified_Date'] = pd.to_datetime(oportunidades['Last_Modified_Date'])
oportunidades['Planned_Delivery_Start_Date'] = pd.to_datetime(oportunidades['Planned_Delivery_Start_Date'])
oportunidades['Planned_Delivery_End_Date'] = pd.to_datetime(oportunidades['Planned_Delivery_End_Date'])
oportunidades_japon = (oportunidades.loc[oportunidades['Region'] == 'Japan'])
oportunidades = (oportunidades.loc[oportunidades['Region'] != 'Japan'])
#oportunidades_japon = oportunidades_japon.iloc[:,:3]
oportunidades_japon['Territory'] = oportunidades_japon['Territory'].replace({'None':'Japan'})
oportunidades = pd.concat([oportunidades, oportunidades_japon], axis=0)
oportunidades['Region'] = oportunidades['Region'].replace({'Japan':'APAC', 'Middle East':'EMEA'})
oportunidades[oportunidades.select_dtypes(['object']).columns] = oportunidades.select_dtypes(['object']).apply(lambda x: x.astype('category'))
#oportunidades = oportunidades.loc[oportunidades['Territory'] != 'None']
#oportunidades = oportunidades.dropna()

filtro_terminos_entrega = oportunidades['Pricing, Delivery_Terms_Quote_Appr']\
     == oportunidades['Pricing, Delivery_Terms_Approved']

filtro_codigo_burocratico = oportunidades['Bureaucratic_Code_0_Approval']\
     == oportunidades['Bureaucratic_Code_0_Approved']

oportunidades = oportunidades.drop(['Pricing, Delivery_Terms_Quote_Appr'\
    ,'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval'\
        , 'Bureaucratic_Code_0_Approved'], axis = 'columns')

oportunidades_posibles = (filtro_terminos_entrega & filtro_codigo_burocratico)

oportunidades.insert(3,'Es_Oportunidad_Posible', oportunidades_posibles)

oportunidades['Es_Oportunidad_Posible'] = oportunidades['Es_Oportunidad_Posible'].replace(
    {True:1, False:0})

oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,...,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A
0,6140,EMEA,Germany,1,0,Bureaucratic_Code_4,2017-07-05,Source_7,Germany,Account_Name_978,...,2019 - 5,Q2,2019,NaT,0,EUR,162240.0,EUR,367419.0,Prod_Category_A_None
1,6146,EMEA,Germany,1,0,Bureaucratic_Code_4,2017-07-05,Source_7,Germany,Account_Name_978,...,2019 - 5,Q2,2019,NaT,0,EUR,78624.0,EUR,367419.0,Prod_Category_A_None
2,6151,EMEA,Germany,1,0,Bureaucratic_Code_4,2017-07-05,Source_7,Germany,Account_Name_978,...,2019 - 5,Q2,2019,NaT,0,EUR,126555.0,EUR,367419.0,Prod_Category_A_None
3,6118,EMEA,Germany,1,0,Bureaucratic_Code_4,2017-07-05,Source_7,Germany,Account_Name_978,...,2019 - 6,Q2,2019,NaT,1,EUR,243360.0,EUR,757783.5,Prod_Category_A_None
4,6124,EMEA,Germany,1,0,Bureaucratic_Code_4,2017-07-05,Source_7,Germany,Account_Name_978,...,2019 - 6,Q2,2019,NaT,0,EUR,157248.0,EUR,757783.5,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2512,15649,APAC,Japan,1,0,Bureaucratic_Code_4,2016-12-09,None,Japan,Account_Name_1039,...,2019 - 5,Q2,2019,NaT,0,JPY,30000.0,JPY,73200.0,Prod_Category_A_None
2513,15655,APAC,Japan,1,0,Bureaucratic_Code_4,2016-12-09,None,Japan,Account_Name_1039,...,2019 - 5,Q2,2019,NaT,0,JPY,43200.0,JPY,73200.0,Prod_Category_A_None
2537,18605,APAC,Japan,1,0,Bureaucratic_Code_4,2013-07-28,Source_3,Japan,Account_Name_1075,...,2020 - 9,Q3,2020,NaT,12,USD,4079928.6,USD,4079928.6,Prod_Category_A_None
2542,5817,APAC,Japan,1,0,Bureaucratic_Code_4,2017-06-30,None,Japan,Account_Name_1542,...,2019 - 7,Q3,2019,NaT,0,JPY,5068800.0,JPY,5068800.0,Prod_Category_A_None


In [458]:
#Encoding para el dataset de testeo, con los mismos que en training set.

encoding_owners = oportunidades.groupby('Opportunity_Owner').agg({'Total_Amount':'mean'})
encoding_owners.columns = ['Encoding_Vendedor']
encoding_owners = encoding_owners.reset_index()
oportunidades = oportunidades.merge(encoding_owners, how='inner', on='Opportunity_Owner')

encoding_territory = oportunidades.groupby('Territory').agg({'Total_Amount' : 'mean'})
encoding_territory.columns = ['Encoding_Territorio']
encoding_territory = encoding_territory.reset_index()
oportunidades = oportunidades.merge(encoding_territory, how='inner', on='Territory')

#encoding_region = oportunidades.groupby('Region').agg({'Total_Amount' : 'mean'})
#encoding_region.columns = ['Encoding_Region']
#encoding_region = encoding_region.reset_index()
#oportunidades = oportunidades.merge(encoding_region, how='inner', on='Region')
oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Region'])],axis='columns')

encoding_moneda = oportunidades.groupby('Total_Amount_Currency').agg({'ASP' : 'mean'})
encoding_moneda.columns = ['Encoding_Moneda']
encoding_moneda = encoding_moneda.reset_index()
oportunidades = oportunidades.merge(encoding_moneda, how='inner', on='Total_Amount_Currency')
oportunidades

oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Bureaucratic_Code'])],axis='columns')

encoding_bill_c = oportunidades.groupby('Billing_Country').agg({'Total_Amount' : 'mean'})
encoding_bill_c.columns = ['Encoding_Billing_Country']
encoding_bill_c = encoding_bill_c.reset_index()
oportunidades = oportunidades.merge(encoding_bill_c, how='inner', on='Billing_Country')
oportunidades

oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Account_Type'])],axis='columns')

oportunidades = pd.concat([oportunidades,pd.get_dummies(oportunidades['Delivery_Terms'])],axis='columns')

encoding_product_family = oportunidades.groupby('Product_Family').agg({'Total_Amount' : 'mean'})
encoding_product_family.columns = ['Encoding_Prod_Family']
encoding_product_family = encoding_product_family.reset_index()
oportunidades = oportunidades.merge(encoding_product_family, how='inner', on='Product_Family')

encoding_account_name = oportunidades.groupby('Account_Name').agg({'Total_Amount' : 'mean'})
encoding_account_name.columns = ['Encoding_Acc_Name']
encoding_account_name = encoding_account_name.reset_index()
oportunidades = oportunidades.merge(encoding_account_name, how='inner', on='Account_Name')

encoding_op_type = oportunidades.groupby('Opportunity_Type').agg({'Total_Taxable_Amount' : 'mean'})
encoding_op_type.columns = ['Encoding_Opportunity_Type']
encoding_op_type = encoding_op_type.reset_index()
oportunidades = oportunidades.merge(encoding_op_type, how='inner', on='Opportunity_Type')

oportunidades['Opportunity_Created_Year'] = oportunidades['Opportunity_Created_Date'].dt.year
oportunidades['Opportunity_Created_Month'] = oportunidades['Opportunity_Created_Date'].dt.month

oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Source '])],axis = 'columns')

oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,...,Source_10,Source_11,Source_13,Source_2,Source_3,Source_4,Source_5,Source_6,Source_7,Source_9
0,6140,EMEA,Germany,1,0,Bureaucratic_Code_4,2017-07-05,Source_7,Germany,Account_Name_978,...,0,0,0,0,0,0,0,0,1,0
1,6118,EMEA,Germany,1,0,Bureaucratic_Code_4,2017-07-05,Source_7,Germany,Account_Name_978,...,0,0,0,0,0,0,0,0,1,0
2,6107,EMEA,Germany,1,0,Bureaucratic_Code_4,2017-07-05,Source_7,Germany,Account_Name_978,...,0,0,0,0,0,0,0,0,1,0
3,6146,EMEA,Germany,1,0,Bureaucratic_Code_4,2017-07-05,Source_7,Germany,Account_Name_978,...,0,0,0,0,0,0,0,0,1,0
4,6124,EMEA,Germany,1,0,Bureaucratic_Code_4,2017-07-05,Source_7,Germany,Account_Name_978,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,11118,EMEA,France,1,0,Bureaucratic_Code_4,2019-02-05,Source_11,France,Account_Name_2036,...,0,1,0,0,0,0,0,0,0,0
2547,12035,APAC,Taiwan,1,0,Bureaucratic_Code_4,2019-03-14,Source_9,Taiwan,Account_Name_800,...,0,0,0,0,0,0,0,0,0,1
2548,12040,APAC,Taiwan,1,0,Bureaucratic_Code_4,2019-03-14,Source_9,Taiwan,Account_Name_800,...,0,0,0,0,0,0,0,0,0,1
2549,12046,APAC,Taiwan,1,0,Bureaucratic_Code_4,2019-03-14,Source_9,Taiwan,Account_Name_800,...,0,0,0,0,0,0,0,0,0,1


In [459]:
#Hacemos el groupby

std_deviation_amounts = oportunidades.groupby('Opportunity_ID').agg({'TRF':'std', 'Opportunity_Created_Month':'std'})
std_deviation_amounts.columns = ['TRF_Std', 'Opp_Created_Month_Std']
std_deviation_amounts = std_deviation_amounts.reset_index()
x_test = oportunidades.groupby('Opportunity_ID').agg({'Opportunity_Created_Year' : 'mean','Opportunity_Created_Month' : 'mean',
                                                      'Encoding_Moneda':'mean', 'Total_Amount': 'sum',
                                                      'Total_Taxable_Amount':'sum','Encoding_Territorio':'mean','Encoding_Vendedor':'mean',\
                                                      'Es_Oportunidad_Posible':'mean', 'TRF':'mean',
                                                       'Encoding_Billing_Country':'mean','Encoding_Prod_Family':'mean', 'Encoding_Acc_Name':'mean',\
                                                      'Encoding_Opportunity_Type':'mean','APAC' : 'mean','Americas':'mean','EMEA':'mean',\
                                                     'Submitted_For_Approval':'mean'})
x_test = x_test.reset_index()
x_test = x_test.merge(std_deviation_amounts, on='Opportunity_ID', how='inner')
x_test = x_test.set_index('Opportunity_ID')
x_test['TRF_Std'] = x_test['TRF_Std'].replace({np.nan:0})
x_test['Opp_Created_Month_Std'] = x_test['Opp_Created_Month_Std'].replace({np.nan:0})
x_test

SpecificationError: nested renamer is not supported

In [ ]:
x_test = x_test.select_dtypes(exclude=['datetime', 'category'])
predictions = xg_reg.predict(x_test)
predictions2 = xg_reg2.predict(x_test)
predictions_final = (predictions+predictions2)/2
predictions_final

In [ ]:
predictions

In [ ]:
predictions2

In [ ]:
x_test = x_test.reset_index()
serie1 = pd.DataFrame(x_test['Opportunity_ID'])
serie2 = pd.DataFrame(predictions)
#serie2 = pd.DataFrame(predictions)
df = pd.DataFrame(serie1)
df['Target'] = serie2
df.columns = ['Opportunity_ID', 'Target']
df

In [ ]:
df.to_csv("Predicciones.csv", index=False)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 7]
xgb.plot_importance(xg_reg)

In [ ]:
xgb.plot_importance(xg_reg2)